In [5]:
%run runaway_functionsv2
# %matplotlib qt


In [14]:
from runaway_functionsv3 import Cluster
cluster = Cluster("Basel_8")
def fast_stars_in_region(cluster):
    sir = cluster.stars_in_region()
    mask_fast2d = sir['v_pec'] > 17.6*u.km/u.s
    mask_vpec3ddosentexist = sir['v_pec3d'].mask
    mask_fast3d = sir['v_pec3d'] > 25*u.km/u.s
    
    return sir[(mask_fast2d & mask_vpec3ddosentexist) | mask_fast3d]
fast_stars_in_region(cluster)

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,e_rmRA,e_rmDE,rRV,e_rRV,v_pec,v_pec3d,e_vpec,e_vpec3d
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,km / s,km / s,km / s,km / s
float64,float64,float64,float64,float64,int32,str10,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord,float64,float64,float32,float32,float64,float32,float64,float64,float64,float64
99.22725858706,7.97823468045,0.0141,0.0136,39.9136,--,733-1788-1,3326053327208023936,1591.49927000,0.5930,0.0173,-1.266,-1.155,0.018,0.014,0.913,4614.2,1.3814,9.252639,2.200538,10.404861,8.204324,49.19,0.16,1548.73706000,1637.15125000,3748792.23428,1598,9.408e+05,1735,4.144e+06,3422,--,--,0.002794,0.003434,0.003884,0.007319,"99.22725858706,7.97823468045,1591.49927",-1.0763750050067902,1.0281250190734863,0.24284768,0.23884769,29.94,1.27,11.228786801637765,31.98,2.705549540248419,1.52
98.69055275937,8.07569472560,0.0356,0.0302,8.1797,--,733-1687-1,3326026144355977472,1458.14001000,0.6601,0.0421,-1.403,0.651,0.039,0.031,1.529,--,--,9.317637,0.329390,9.419267,9.089877,--,--,1378.07080000,1536.19507000,3530953.30802,1620,2.332e+06,5770,1.833e+06,2216,--,--,0.0028,0.003873,0.004001,0.007874,"98.69055275937,8.0756947256,1458.14001",-1.2133750050067902,2.834125019073486,0.26384768,0.2558477,--,--,21.30802513378318,--,5.478361559307416,--
98.49450273357,7.75882325570,0.0234,0.0244,17.1786,--,--,3325966637588331904,1422.33301000,0.6829,0.0311,0.397,-1.004,0.029,0.025,1.989,4636.4,1.8177,10.560462,1.862149,11.482549,9.620399,71.39,2.28,1364.09021000,1480.81653000,1123989.10264,310.9,3.487e+05,336.8,1.125e+06,398.9,--,--,0.002772,0.002981,0.003799,0.00678,"98.49450273357,7.7588232557,1422.33301",0.5866249949932099,1.1791250190734863,0.2538477,0.2498477,52.14,3.39,8.878965392361131,52.89,2.225480673031585,3.36
98.91793033140,8.40607900769,0.0277,0.0232,30.7213,--,--,3326133660275716480,1192.20020000,0.8168,0.0316,3.212,0.952,0.030,0.024,1.821,--,--,11.122290,1.406081,11.761770,10.355689,50.24,0.77,1148.89038000,1250.99280000,669932.02689,165.4,2.696e+05,163.5,5.713e+05,246.5,--,--,0.002768,0.002867,0.003808,0.006675,"98.9179303314,8.40607900769,1192.2002",3.40162499499321,3.1351250190734863,0.25484768,0.2488477,30.99,1.88,26.141773209726033,40.54,6.590592675179801,4.49
98.80943694927,8.02405631161,0.0143,0.0132,14.9497,--,733-1827-1,3325977426545923840,1605.87500000,0.6013,0.0165,2.491,-3.233,0.018,0.016,0.950,--,--,11.189508,1.873466,12.116813,10.243346,54.60,0.20,1569.39270000,1654.23608000,629714.19999,137.1,1.944e+05,153.6,6.336e+05,217.9,--,--,0.002765,0.002919,0.003798,0.006717,"98.80943694927,8.02405631161,1605.875",2.68062499499321,-1.0498749809265138,0.24284768,0.24084769,35.35,1.31,21.913645483039886,41.59,5.315870165917745,3.01
98.32807784189,7.85959204576,0.0283,0.0251,17.3699,--,733-2008-1,3325994876995037824,1207.66919000,0.8004,0.0329,-0.988,-7.367,0.032,0.025,1.391,5133.8,2.7982,11.342217,1.351886,11.946281,10.594396,-26.32,1.19,1168.05396000,1258.94336000,547091.22454,125.6,2.275e+05,103.9,4.586e+05,203.2,--,--,0.002767,0.002834,0.00381,0.006644,"98.32807784189,7.85959204576,1207.66919",-0.7983750050067902,-5.183874980926514,0.25684768,0.2498477,-45.57,2.30,30.02419190903883,54.57,7.506411329747386,4.55
98.15627576457,8.56722476217,0.0173,0.0162,39.7161,--,--,3326255495610553728,1175.12512000,0.8252,0.0189,-1.074,4.618,0.020,0.017,1.133,4952.4,2.7904,11.348566,1.440937,12.007280,10.566343,40.51,0.21,1149.72412000,1201.94348000,54390

In [2]:
from astroquery.simbad import Simbad
Simbad.query_object(objectname)

hi


In [25]:
cluster = Cluster('Alessi_43')
# runCode(cluster)
plot_cmd(cluster)

Original Data
2 memebrs do not have BP-RP from Dias.
144/240 members matched in dr3 search region,81 have > 17 Gmag
144/144 members have good parallax
File ./Clusters/Alessi_43/Alessi_43_runaways.tsv not present.
iso_4_temp0 Av:0.912, logage:6.763, FeH:-0.051
iso_4_temp Av:0.912, logage:6.763, FeH:-0.051
calculating temperatures based on: {}


<AxesSubplot: title={'center': 'CMD for Alessi_43'}, xlabel='$G_{BP}-G_{RP}$ (mag)', ylabel='$G$ (mag)'>

In [24]:
ra =cluster.runaways_all
ra[ra['Source']==5524355337493304832]


RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,v_pec,Temp. Est
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc",mas / yr,mas / yr,km / s,K
float64,float64,float64,float64,float64,int32,str10,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord,float64,float64,float64,float64
133.50740483154,-42.13832490675,0.0132,0.0149,44.7298,--,--,5524355337493304832,927.84228500,1.0446,0.0172,9.759,-16.466,0.018,0.019,1.050,15014.8,4.5993,14.726832,1.661750,15.506360,13.844610,31.15,5.81,913.74408000,946.44604500,24222.21574,8.685,8568,18.94,2.298e+04,25.09,--,--,0.002783,0.00368,0.003961,0.007641,"133.50740483154,-42.13832490675,927.842285",15.289720710754395,-20.38591925239563,112.07163078207608,15014.8


In [6]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize
%matplotlib qt
# Assuming `ti` is already defined
ti = theoretical_isochrone(cluster, logage=7.1, Av=2.9, output='table')

# Preparing the data
x = ti['Teff0']
y = ti['Gmag']

# Scatter points data
scatter_temperatures = np.array([24000, 21000])
scatter_magnitudes = np.array([9.840, 9.470])

# Combine temperatures for normalization
all_temperatures = np.concatenate([x, scatter_temperatures])

# Create normalization based on the combined temperature data
norm = Normalize(vmin=all_temperatures.min(), vmax=all_temperatures.max())

# Creating segments for the line plot
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# Creating the LineCollection
lc = LineCollection(segments, cmap='viridis', norm=norm)
lc.set_array(x)
lc.set_linewidth(2)

# Plotting the data
fig, ax = plt.subplots()
ax.add_collection(lc)

# Setting the limits to ensure all data is visible
all_magnitudes = np.concatenate([y, scatter_magnitudes])

ax.set_xlim(all_temperatures.max(), all_temperatures.min())  # Invert x-axis
ax.set_ylim(all_magnitudes.max(), all_magnitudes.min())  # Invert y-axis
ax.set_xscale('log')

# Adding a color bar to indicate the color mapping
cbar = plt.colorbar(lc)
cbar.set_label('Teff0')

# Adding the scatter points colored according to their temperature with the same normalization
scatter = ax.scatter(scatter_temperatures, scatter_magnitudes, c=scatter_temperatures, cmap='viridis', norm=norm, edgecolor='black', zorder=5)

# Adding labels next to the scatter points
for temp, mag in zip(scatter_temperatures, scatter_magnitudes):
    ax.annotate(f'{temp} K', (temp, mag), textcoords="offset points", xytext=(5, 5), ha='center', color='black')

# Adding labels and title for clarity
plt.xlabel('Teff0 (log scale)')
plt.ylabel('Gmag')
plt.title('Color-Magnitude Diagram')

# Displaying the plot
plt.show()


Berkeley_97 (Changed)
Av: 2.539 --> 2.9
logage: 7.589 --> 7.1
[Fe/H]: -0.1 --> -0.1
Theoretical isochrone exists, reading it.


In [11]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize

# Function to compute theoretical isochrone
def theoretical_isochrone(logage, Av):
    # Replace this with your actual function call
    # ti = theoretical_isochrone(cluster, logage=logage, Av=Av, output='table')
    
    # Dummy implementation for illustration
    teff = np.linspace(5000, 20000, 100)
    gmag = 5 * np.log10(teff) - 10
    ti = {'Teff0': teff, 'Gmag': gmag}
    
    return ti

# Define parameters to vary
logage_values = [7.0, 7.1, 7.2]  # Example values for logage
Av_values = [2.5, 2.9, 3.3]     # Example values for Av

# Prepare the plot
fig, ax = plt.subplots()

# Create a colormap and normalization instance
cmap = plt.cm.get_cmap('viridis')
norm = Normalize(vmin=min(logage_values), vmax=max(logage_values))

# Plot each isochrone with varying parameters
for logage, Av in zip(logage_values, Av_values):
    # Compute theoretical isochrone
    ti = theoretical_isochrone(logage, Av)
    
    # Extract data
    x = ti['Teff0']
    y = ti['Gmag']
    
    # Create segments for the line plot
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    # Create LineCollection with appropriate color based on logage
    lc = LineCollection(segments, cmap=cmap, norm=norm)
    lc.set_array(np.array([logage] * len(x)))
    lc.set_linewidth(2)
    
    # Add LineCollection to the plot
    ax.add_collection(lc)

# Add color bar to indicate logage
cbar = plt.colorbar(lc)
cbar.set_label('log(age)')
cbar.set_ticks(logage_values)

# Invert y-axis for magnitude
ax.invert_yaxis()

# Set labels and title
plt.xlabel('Teff0')
plt.ylabel('Gmag')
plt.title('Theoretical Isochrones Comparison')

# Show plot
plt.show()


In [59]:
plot_cmd(cluster, logage=7.1,Av=2.9,multiple=1)

In [55]:
# reRunCode(cluster, separation_factor=0.5)
runCode(cluster,logage=7,Av=3)

=======================================Berkeley_97==================================================
All tables present (stars_in_region,stars_fromDR3,stars_fromDR3_dis,fs,nsfs)
----------------------------Berkeley_97: traceback--------------------------------------------------
Berkeley_97/Berkeley_97_runaways_all.tsv exists. Skipping traceback and temp estimates.
257 stars had been checked, 6 traced back to cluster
----------Berkeley_97: runaways and isochrone plot--------------------------------------------------
0 memebrs do not have BP-RP from Dias.
35/58 members matched in dr3 search region,14 have > 17 Gmag
27/35 members have good parallax
fits image exists in Berkeley_97 folder


1 Runaway(s) found Temp. Est    RA_ICRS_1       DE_ICRS_1    ...   e_RPmag     e_BP-RP  
    K           deg             deg       ...                        
--------- --------------- --------------- ... ----------- -----------
  13630.0 339.89581711423  58.99547917199 ...    0.004229    0.008081


In [4]:
l = [1,2,3,4,5]
l1 = [3,4]
l+l1

[1, 2, 3, 4, 5, 3, 4]

In [34]:
cluster.read_table('runaways_all')

Temp. Est,RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,v_pec,µ_pec,rmRA,e_rmRA,rmDE,e_rmDE,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP
K,deg,deg,mas,mas,,,,,pc,mas,mas,km / s,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,
float64,float64,float64,float64,float64,float64,int32,str1,int64,float64,float64,float32,float64,float64,float64,float32,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32
9288.0,339.89581711423,58.99547917199,0.0202,0.0197,1.5205,--,--,2008384504995669248,2620.36230000,0.3560,0.0241,29.876,2.405,2.349,0.122,0.517,0.117,-0.379,-1.352,0.025,0.024,1.086,9168.0,3.7061,15.384143,1.182070,15.883185,14.701116,--,--,2461.53345000,2759.61401000,13221.76503,3.482,6055,14.81,1.044e+04,18.25,--,--,0.00277,0.003852,0.004229,0.008081
8439.0,339.92937723020,59.00404480063,0.0142,0.0142,2.4555,--,--,2008384573715370624,2923.95410000,0.3111,0.0174,22.147,1.598,-1.563,0.115,-0.333,0.111,-4.291,-2.202,0.018,0.018,1.104,9302.7,3.5455,14.625331,1.209811,15.134336,13.924525,--,--,2765.68970000,3111.05029000,26595.89065,6.887,1.207e+04,14.3,2.135e+04,25.38,--,--,0.00277,0.003073,0.003994,0.007066
8439.0,340.11961278418,58.99915127691,0.0159,0.0166,8.3403,--,--,2008381034662369920,2768.34961000,0.3392,0.0194,32.542,2.480,2.281,0.118,0.973,0.114,-0.447,-0.896,0.021,0.021,1.142,9221.7,3.4526,14.785446,1.218026,15.304078,14.086052,--,--,2622.75366000,2950.88965000,22949.24853,4.602,1.032e+04,11.25,1.84e+04,13.88,--,--,0.002764,0.003031,0.003867,0.006898
7610.0,339.82327510265,59.02791551662,0.0373,0.0353,1.6227,--,--,2008430306528315136,2914.14893000,0.3154,0.0444,21.453,1.553,-1.036,0.142,-1.157,0.136,-3.764,-3.026,0.045,0.043,1.051,--,--,16.566925,1.372125,17.177988,15.805863,--,--,2617.74390000,3300.76758000,4448.12149,2.398,1838,8.274,3774,7.525,--,--,0.002817,0.005629,0.004356,0.009985
7517.0,339.56494819769,58.99344914887,0.0102,0.0102,8.8317,--,--,2008407491661705216,2915.46533000,0.3094,0.0119,33.943,2.456,-1.604,0.110,-1.860,0.106,-4.332,-3.729,0.013,0.013,1.234,6899.3,2.8208,13.123951,1.392846,13.738716,12.345870,--,--,2842.50513000,3028.43872000,106014.76839,30.59,4.364e+04,44.13,9.137e+04,21.25,--,--,0.002773,0.002998,0.003788,0.006786
7492.0,339.96150942069,59.05945190536,0.0364,0.0338,4.7645,--,--,2008385329629591424,2716.07910000,0.3593,0.0427,27.484,2.135,-1.785,0.140,-1.171,0.137,-4.513,-3.040,0.043,0.044,0.975,9604.5,4.0106,16.551115,1.518426,17.218628,15.700202,--,--,2433.93335000,3059.97217000,4513.36761,2.723,1770,15.18,4160,12.18,--,--,0.002832,0.009721,0.004938,0.01466
7367.0,339.55127558593,58.91799587450,0.0190,0.0186,10.6001,--,--,2008406185991730560,2902.17993000,0.3132,0.0227,32.178,2.339,-2.272,0.122,-0.557,0.116,-5.000,-2.426,0.025,0.023,1.006,8971.7,3.5187,15.297109,1.432618,15.936389,14.503771,--,--,2719.46582000,3068.63403000,14325.28459,5.592,5766,15.75,1.252e+04,14.6,--,--,0.002788,0.004072,0.003986,0.008058
7306.0,339.57212568342,59.04007680090,0.0354,0.0334,8.8388,--,--,2008433978715406336,2920.62378000,0.3214,0.0407,32.574,2.353,-1.939,0.141,-1.333,0.136,-4.667,-3.202,0.044,0.043,1.001,--,--,16.514180,1.461832,17.176619,15.714787,--,--,2555.96191000,3251.21265000,4669.55275,2.276,1840,10.76,4104,8.21,--,--,0.002806,0.006938,0.004359,0.0113
